In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Trich xuat cac dac trung can su dung va chuyen doi ve dang numpy de xu ly
import numpy as np
def extract_feature(data, feature_titles):
    features = [data[val].values for val in feature_titles]
    
    return np.stack(features, axis= - 1)
    
    

In [4]:
# reshape ve 1 cot
a = np.random.rand(2,3)
b= np.reshape(a, [-1, 1])
b.shape

(6, 1)

In [5]:
# reshape ve 1 dong
a = np.random.rand(2,3)
b= np.reshape(a, [1, -1])
b.shape

(1, 6)

In [ ]:
# ve phan phoi cua mot bien
fig = plt.figure()
ax = fig.add_subplot(111)
#Bieu do tan suat
data['GrLivArea'].hist(bins=50, ax=ax, density=True, color='red')
#Bieu do mat do
data['GrLivArea'].plot.density(color='red')
#Bieu do mat do
X_train['Age_mean'].plot(kind='kde', ax=ax, color='green')
#boxplot
X_train[["GarageYrBlt", 'GarageYrBlt_median']].boxplot()
# gan label cho hinh
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')



In [ ]:
#mean-median-imputer
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(X_train)
imputer.transform(X_test)

from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer(transformers=[
    ("mean_imputer",SimpleImputer(strategy='mean'),_list_cac_cot_dung)
    ("median_imputer",SimpleImputer(strategy="median"), list_cac_cot_dung)
],remainder="passthrough")
preprocessing.fit(X_train)
#phuong thuc
preprocessor.transformers

preprocessing.named_transformers_['mean_imputer'].statistics_

preprocessing.transform(X_train)

np.mean(np.isnan(X_train)) # boi vi sklearn chuyen dataframe ve numpy

preprocessor.transformers_

remainder_cols = [cols_to_use[c] for c in [0, 1, 2, 3, 4, 5]]
remainder_cols

from sklearn.impute import SimpleImputer, MissingIndicator
indicator = MissingIndicator(error_on_new=True, features="missing-only")

# we can find the feature names by passing the index to the
# list of columns

X_train.columns[indicator.features_]

tmp = indicator.transform(X_train)

indicator_cols = [c+'_NA' for c in X_train.columns[indicator.features_]]

# and now we concatenate
X_train = pd.concat([
    X_train.reset_index(),
    pd.DataFrame(tmp, columns = indicator_cols)],
    axis=1)

X_train.head()

In [ ]:
features_categorical = [c for c in data.columns if data[c].dtypes=='O']

# find numerical variables
# those different from object and also excluding the target SalePrice
features_numerical = [c for c in data.columns if data[c].dtypes!='O' and c !='SalePrice']

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
numeric_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()))
])
categorical_transformer = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy='constant',fill_value="missing"))
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformer = [ 
    ("numerical",numeric_transformer,feature_numerical),
    ("caterorical",categorical_transformer, features_categorical)
])

param_grid = {
    "preprocessor__numerical__imputer__strategy" : ['mean','median'],
    "preprocessor__categorical__imputer__strategy":['most_frequent',"constant"],
    'regressor__alpha':[10,100,200],
}
clf = Pipeline(steps =[
    ('preprocessor',preprocessor),
    ("regressor",Lasso(max_iter=2000))
    ])

from sklearn.model_selection import train_test_split, GridSearchCV
grid_search = GridSearchCV(clf, param_grid,cv=5,iid=False,n_jobs=-1,scoring='r2')

grid_search.fit(X_train, y_train)

print(("best linear regression from grid search: %.3f"
       % grid_search.score(X_train, y_train)))

grid_search.cv_results_["mean_test_score"]

encoder = OneHotEncoder(categories='auto',
                       drop='first', # trả về k-1, sử dụng drop=false để trả về k biến giả
                       sparse=False,
                       handle_unknown='error') # giúp xử lý nhãn hiếm

encoder.fit(X_train.fillna('Missing'))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(X_train['Neighborhood'])

In [ ]:
d = defaultdict(LabelEncoder)
train_transformed = X_train.apply(lambda x: d[x.name].fit_transform(x))

# # Using the dictionary to encode future data
test_transformed = X_test.apply(lambda x: d[x.name].transform(x))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
top_10 = [
    x for x in X_train['Neighborhood'].value_counts().sort_values(ascending=False).head(10).index
]

for label in top_10:
    ## VIẾT CODE Ở ĐÂY:
    X_train['Neighborhood' + '_' + label] = np.where(
        X_train['Neighborhood'] == label, 1, 0)
    
    X_test['Neighborhood' + '_' + label] = np.where(X_test["Neighborhood"]==label,1,0)


In [ ]:
def calculate_top_categories(df, variable, how_many=10):
    return [
        x for x in df[variable].value_counts().sort_values(
            ascending=False).head(how_many).index
    ]


def one_hot_encode(train, test, variable, top_x_labels):

    for label in top_x_labels:
        train[variable + '_' + label] = np.where(
            train[variable] == label, 1, 0)
        
        test[variable + '_' + label] = np.where(
            test[variable] == label,1, 0)

In [ ]:
## Yêu cầu 5
# thiết lập StandardScaler để nó loại bỏ mean
# nhưng không chia cho độ lêch chuẩn

## VIẾT CODE Ở ĐÂY:
scaler_mean = StandardScaler(with_mean=True, with_std=False)

# thiết lập robustscaler để nó KHÔNG loại median
# nhưng chuẩn hóa bằng max()-min(), quan trọng với
# phạm vi quantile từ 0-100, thể hiện min và max

## VIẾT CODE Ở ĐÂY:
scaler_minmax = RobustScaler(with_centering=False,
                             with_scaling=True,
                             quantile_range=(0, 100))

# khớp scaler với tập huấn luyện để nó học các tham số
## VIẾT CODE Ở ĐÂY:
scaler_mean.fit(X_train)
scaler_minmax.fit(X_train)

# biến đổi tập huấn luyện và tập kiểm tra
## VIẾT CODE Ở ĐÂY:
X_train_scaled = scaler_minmax.transform(scaler_mean.transform(X_train))
X_test_scaled = scaler_minmax.transform(scaler_mean.transform(X_test))

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # tập hợp tất cả tên của các cột tương quan tập hợp tất cả tên của các cột tương quan
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # cần quan tâm tới các giá trị hệ số tuyệt đối 
                colname = corr_matrix.columns[i]  # lấy tên cột 
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
sfs = SFS(RandomForestRegressor(n_estimators=10, n_jobs=4, random_state=10), 
           k_features=20, 
           forward=False, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS


In [ ]:
from sklearn.feature_selection import VarianceThreshold, f_classif, SelectKBest

In [2]:
import numpy as np 
a = np.random.randn(90,92)
b = np.random.randn(92,1)
c = np.dot(a,b)
print(a.shape[::-1])

(92, 90)


In [8]:
a x b

SyntaxError: invalid syntax (<ipython-input-8-a667d480eddd>, line 1)

In [9]:
print(c.shape)

(90, 1)


In [15]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two'],
                   'C' : [1, 5, 5, 2, 5, 5],
                   'D' : [4, 5., 8., 1., 2., 9.]})

In [16]:
df["D"].map(np.sqrt)

0    2.000000
1    2.236068
2    2.828427
3    1.000000
4    1.414214
5    3.000000
Name: D, dtype: float64

In [11]:
df.groupby(["A"]).transform(lambda x: (x - x.mean()))
df[]

<ipython-input-11-8e2f524d6674>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  df.groupby(["A"]).transform(lambda x: (x - x.mean()))


,C,D
0,-2.666667,NaN
1,1.000000,0.0
2,1.333333,3.0
3,-2.000000,-4.0
4,1.333333,-3.0
5,1.000000,4.0


In [12]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
from feature_engine.selection import DropDuplicateFeatures
sel = DropDuplicateFeatures()
sel.fit(X_4)
X_5 = sel.transform(X_4)

In [ ]:
from feature_engine.selection import DropCorrelatedFeatures, SmartCorrelatedSelection
sel = DropCorrelatedFeatures(
    threshold=0.8,
    method='pearson',
    missing_values='ignore'
)


# find correlated features

sel.fit(X_train)
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats as stats
from feature_engine.selection import DropCorrelatedFeatures
from feature_engine.selection import DropDuplicateFeatures
from feature_engine.selection import RecursiveFeatureAddition
from feature_engine.selection import RecursiveFeatureElimination
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor, SGDClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectFromModel, SelectKBest
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler,  OneHotEncoder, OrdinalEncoder,PolynomialFeatures,PowerTransformer, FunctionTransformer
from sklearn.svm import SVC, SVR 


In [ ]:
from sklearn.metrics import r2_score, roc_auc_score, mean_squared_error, accuracy_score,confusion_matrix, f1_score
